In [0]:
from zipfile import ZipFile
import os


In [39]:
%tensorflow_version 2.x


from __future__ import print_function

import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten


TensorFlow is already loaded. Please restart the runtime to change versions.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
DATADIR = "/content/drive/My Drive/data"
filepath = os.path.join(DATADIR, 'dogs-vs-cats.zip')
with ZipFile(filepath, 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall('./temp')

In [0]:
with ZipFile("temp/test1.zip", 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall('./data')

In [0]:
with ZipFile("temp/train.zip", 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall('./data')

In [0]:
DATA_DIR = "data"
TRAIN_DIR = "train"
TEST_DIR = "test1"


In [0]:
filenames = os.listdir(os.path.join(DATA_DIR, TRAIN_DIR))
labels = [filename.split(".")[0] for filename in filenames]

from sklearn.preprocessing import LabelEncoder
lbl = LabelEncoder()
labels = lbl.fit_transform(labels)

In [21]:
lbl.classes_

array(['cat', 'dog'], dtype='<U3')

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(filenames, labels, stratify=labels, test_size=0.2)

In [9]:
len(X_train), len(X_val) 

(20000, 5000)

In [0]:
os.mkdir("data/train/dog")
os.mkdir("data/train/cat")
os.mkdir("data/val")
os.mkdir("data/val/dog")
os.mkdir("data/val/cat")

In [0]:
DATA_DIR = "data"
TRAIN_DIR = "train"
VAL_DIR = "val"

import shutil
for filename, label in zip(X_train,y_train):
  source_file = os.path.join(DATA_DIR, TRAIN_DIR, filename)
  target_file = os.path.join(DATA_DIR, TRAIN_DIR, 
                             lbl.inverse_transform([label])[0], filename)
  shutil.move(source_file, target_file)

for filename, label in zip(X_val,y_val):
  source_file = os.path.join(DATA_DIR, TRAIN_DIR, filename)
  target_file = os.path.join(DATA_DIR, VAL_DIR, 
                             lbl.inverse_transform([label])[0], filename)
  shutil.move(source_file, target_file)


In [16]:
len(os.listdir(
    os.path.join(DATA_DIR, TRAIN_DIR, "dog"))), len(os.listdir(
    os.path.join(DATA_DIR, TRAIN_DIR, "cat"))),

(10000, 10000)

In [17]:
len(os.listdir(
    os.path.join(DATA_DIR, VAL_DIR, "dog"))), len(os.listdir(
    os.path.join(DATA_DIR, VAL_DIR, "cat"))),

(2500, 2500)

In [0]:
def _preprocess_image(image):   
    image = tf.image.decode_jpeg(image)   
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, [128, 128])
    image /= 255.0  # normalize to [0,1] range    
    return image

def _load_and_preprocess_image(path):   
    image = tf.io.read_file(path)
    image = _preprocess_image(image)
    return image

def load_and_preprocess_from_path_label(path, label):   
    return _load_and_preprocess_image(path), label  

def get_dataset(ds):
    image_label_ds = ds.map(load_and_preprocess_from_path_label)
    image_label_ds = image_label_ds.shuffle(1024)
    image_label_ds = image_label_ds.batch(1024)
    image_label_ds = image_label_ds.prefetch(buffer_size=1024)
    return image_label_ds

In [0]:
import random

In [0]:
X_train = os.listdir(os.path.join(DATA_DIR, TRAIN_DIR, "dog")) + os.listdir(os.path.join(DATA_DIR, TRAIN_DIR, "cat"))
random.shuffle(X_train)
y_train = [0 if filename.split(".")[0] == "cat" else 1 for filename in X_train ] 
X_train = [os.path.join(DATA_DIR, TRAIN_DIR, "cat",filename) \
           if filename.split(".")[0] == "cat" \
           else os.path.join(DATA_DIR, TRAIN_DIR, "dog",filename) 
           for filename in X_train ]

In [0]:
X_val = os.listdir(os.path.join(DATA_DIR, VAL_DIR, "dog")) + os.listdir(os.path.join(DATA_DIR, VAL_DIR, "cat"))
random.shuffle(X_val)
y_val = [0 if filename.split(".")[0] == "cat" else 1 for filename in X_val ] 
X_val = [os.path.join(DATA_DIR, VAL_DIR, "cat",filename) \
           if filename.split(".")[0] == "cat" \
           else os.path.join(DATA_DIR, VAL_DIR, "dog",filename) 
           for filename in X_val ]

In [0]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))

train_dataset = get_dataset(train_dataset)
val_dataset = get_dataset(val_dataset)


In [0]:
model = Sequential()
model.add(Flatten(input_shape = (128, 128,3)))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.summary()
model.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

history = model.fit(train_dataset, epochs=50, validation_data=val_dataset,
                    verbose=1, shuffle=True)

score = model.evaluate(test_dataset, verbose=0)

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_7 (Flatten)          (None, 49152)             0         
_________________________________________________________________
dense_49 (Dense)             (None, 128)               6291584   
_________________________________________________________________
dense_50 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_51 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_52 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_53 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_54 (Dense)             (None, 64)               